# Import and unzip LST21 corpus

In [ ]:
import os
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
!unzip /content/drive/MyDrive/POSTAG/Data/LST21/may_wiki_0-50_pos.zip -d ./may_wiki_0-50_pos/
!unzip /content/drive/MyDrive/POSTAG/Data/LST21/may_wiki_50-100_pos.zip -d ./may_wiki_50-100_pos/
!unzip /content/drive/MyDrive/POSTAG/Data/LST21/may_wiki_100-150_pos.zip -d ./may_wiki_100-150_pos/
!unzip /content/drive/MyDrive/POSTAG/Data/LST21/may_wiki_150-200_pos.zip -d ./may_wiki_150-200_pos/
!unzip /content/drive/MyDrive/POSTAG/Data/LST21/may_wiki_200-250_pos.zip -d ./may_wiki_200-250_pos/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# Install important package (simple tranformer + wandb)

In [ ]:
!pip install -q simpletransformers

In [ ]:
!pip install -q datasets transformers[sentencepiece] simpletransformers

In [ ]:
!pip install wandb

In [ ]:
!wandb login 28d2de59b2adb6064f3125b6c41e3914823d3cbd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import logging
import torch
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd

# Create dataset

In [ ]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

**Create Train set**

In [ ]:
# create train set
listfile = os.listdir('/content/may_wiki_200-250_pos/content/may_wiki_200-250_pos/')

In [ ]:
len(listfile)

25550

In [ ]:
#listfilecut = listfile[:len(listfile)//12].copy()

In [ ]:
#len(listfilecut)

In [ ]:
path_to_text = '/content/may_wiki_200-250_pos/content/may_wiki_200-250_pos/'

sentence_id = []
words = []
labels = []
sent_num = 0
for file_ in tqdm(listfile): ################### listfile
  #print(file_)
  try:
    f = open(path_to_text+file_)

    for line in f:
      if len(line)==1 or line.startswith('-DOCSTART') or line[0]=="\n": #ถ้าเจอบรรทัดว่าง
        sentence_id.append('')
        words.append('')
        labels.append('')
        sent_num += 1
      else:
        splits = line.split('\t')
        sentence_id.append(sent_num)
        words.append(splits[0])
        labels.append(splits[1])
    f.close()
  except:
    #print("Directory")
    pass


train_ = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels})

  0%|          | 0/25550 [00:00<?, ?it/s]

In [ ]:
train_ = train_.drop(train_[train_.sentence_id == ''].index)

In [ ]:
train_

,sentence_id,words,labels
0,0,สุลตานะห์,NN
1,0,เมเลก,NN
2,0,_,PU
3,0,ตูร์,NN
4,0,ฮาน,NN
...,...,...,...
10311857,138967,ครั่ง,NN
10311858,138967,น้อย,VV
10311860,138968,พ.ศ.,NN
10311861,138968,_,PU


In [ ]:
############# read frome csv ##########
train_ = pd.read_csv('/content/drive/MyDrive/POSTAG/Data/LSTdata_all.csv', dtype={'sentence_id': int, 'words': 'str' , 'labels':'str'})

#train_ = pd.read_csv('/content/drive/MyDrive/POSTAG/Data/LSTdata_all.csv')

ValueError: ignored

In [ ]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50431532 entries, 0 to 50431531
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   sentence_id  object
 1   words        object
 2   labels       object
dtypes: object(3)
memory usage: 1.1+ GB


,sentence_id,words,labels
50431527,691216,ตัดเย็บ,VV
50431528,691216,เสื้อ,NN
50431529,691216,โหล,NN
50431530,691216,จำนวน,NN
50431531,691216,มาก,VV


In [ ]:
train_= train_.drop(train_[train_.sentence_id == ''].index)

In [ ]:
train_['sentence_id'] = train_['sentence_id'].astype('int64')

ValueError: ignored

In [ ]:
train_ = train_.drop(columns='Unnamed: 0')

In [ ]:
arr  = train_['sentence_id'].unique()

In [ ]:
arr

array([0, 1, 2, ..., 691214, 691215, 691216], dtype=object)

In [ ]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50431532 entries, 0 to 50431531
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   sentence_id  object
 1   words        object
 2   labels       object
dtypes: object(3)
memory usage: 1.1+ GB


**Make a very big data frame**

In [ ]:
sent_num = 0

In [ ]:
def make_dataframe(listfile , path_to_text):
  global sent_num
  sentence_id = []
  words = []
  labels = []
  for file_ in tqdm(listfile):
    #print(file_)
    try:
      f = open(path_to_text+file_)
      for line in f:
        if len(line)==1 or line.startswith('-DOCSTART') or line[0]=="\n": #ถ้าเจอบรรทัดว่าง
          sentence_id.append('')
          words.append('')
          labels.append('')
          sent_num += 1
        else:
          splits = line.split('\t')
          sentence_id.append(int(sent_num))
          words.append(splits[0])
          labels.append(splits[1])
      f.close()
    except:
      print("Directory")
      pass
  df = pd.DataFrame({ "sentence_id": sentence_id ,"words": words, "labels": labels})
  df = df.drop(df[df.sentence_id == ''].index)
  df['sentence_id'] = df['sentence_id'].astype('int64')

  return df

In [ ]:
data1 = os.listdir('/content/may_wiki_0-50_pos/content/may_wiki_0-50_pos/')
data1_path = '/content/may_wiki_0-50_pos/content/may_wiki_0-50_pos/'

data2 = os.listdir('/content/may_wiki_100-150_pos/content/may_wiki_100-150_pos/')
data2_path = '/content/may_wiki_100-150_pos/content/may_wiki_100-150_pos/'

data3 = os.listdir('/content/may_wiki_150-200_pos/content/may_wiki_150-200_pos/')
data3_path = '/content/may_wiki_150-200_pos/content/may_wiki_150-200_pos/'

data4  = os.listdir('/content/may_wiki_200-250_pos/content/may_wiki_200-250_pos/')
data4_path = '/content/may_wiki_200-250_pos/content/may_wiki_200-250_pos/'

data5 = os.listdir('/content/may_wiki_50-100_pos/content/may_wiki_50-100_pos/')
data5_path = '/content/may_wiki_50-100_pos/content/may_wiki_50-100_pos/'

In [ ]:
data1_df = make_dataframe(data1 , data1_path)
data2_df = make_dataframe(data2 , data2_path)
data3_df = make_dataframe(data3 , data3_path)
data4_df = make_dataframe(data4 , data4_path)
data5_df = make_dataframe(data5 , data5_path)

  0%|          | 0/25011 [00:00<?, ?it/s]

Directory


  0%|          | 0/25550 [00:00<?, ?it/s]

  0%|          | 0/25510 [00:00<?, ?it/s]

  0%|          | 0/25550 [00:00<?, ?it/s]

  0%|          | 0/25556 [00:00<?, ?it/s]

Directory


In [ ]:
full_train_data = pd.concat([data1_df, data2_df,data3_df,data4_df,data5_df], ignore_index=True)

In [ ]:
full_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50431528 entries, 0 to 50431527
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   sentence_id  int64 
 1   words        object
 2   labels       object
dtypes: int64(1), object(2)
memory usage: 1.1+ GB


**Create Validation Set**

In [ ]:
#create val set

In [ ]:
!unzip /content/drive/MyDrive/POSTAG/Data/LST20_Corpus.zip -d ./

In [ ]:
listfile = os.listdir('/content/LST20_Corpus/train/')
path_to_text = '/content/LST20_Corpus/train/'

sentence_id = []
words = []
labels = []
sent_num = 0
for file_ in tqdm(listfile):
  #print(file_)
  try:
    f = open(path_to_text+file_)
    for line in f:
      if len(line)==1 or line.startswith('-DOCSTART') or line[0]=="\n": #ถ้าเจอบรรทัดว่าง
        sentence_id.append('')
        words.append('')
        labels.append('')
        sent_num += 1
      else:
        splits = line.split('\t')
        sentence_id.append(sent_num)
        words.append(splits[0])
        labels.append(splits[1])
    f.close()
  except:
    #print("Directory")
    pass


train_ = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels})

  0%|          | 0/7589 [00:00<?, ?it/s]

In [ ]:
train_ = train_.drop(train_[train_.sentence_id == ''].index)

In [ ]:
train_

,sentence_id,words,labels
0,0,ตร.,NN
1,0,เรียก,VV
2,0,อดีต,NN
3,0,สาว,NN
4,0,ใช้,VV
...,...,...,...
2778030,63309,"""",PU
2778031,63309,_,PU
2778032,63309,นาย,NN
2778033,63309,สุรวุฒิ,NN


In [ ]:
listfile = os.listdir('/content/LST20_Corpus/eval/')
path_to_text = '/content/LST20_Corpus/eval/'

sentence_id = []
words = []
labels = []
sent_num = 0
for file_ in tqdm(listfile):
  #print(file_)
  try:
    f = open(path_to_text+file_)
    for line in f:
      if len(line)==1 or line.startswith('-DOCSTART') or line[0]=="\n": #ถ้าเจอบรรทัดว่าง
        sentence_id.append('')
        words.append('')
        labels.append('')
        sent_num += 1
      else:
        splits = line.split('\t')
        sentence_id.append(sent_num)
        words.append(splits[0])
        labels.append(splits[1])
    f.close()
  except:
    #print("Directory")
    pass


eval_ = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels})

  0%|          | 0/949 [00:00<?, ?it/s]

In [ ]:
eval_ = eval_.drop(eval_[eval_.sentence_id == ''].index)

In [ ]:
eval_

,sentence_id,words,labels
0,0,บาร์เรน,NN
1,0,สนใจ,VV
2,0,ให้,CC
3,0,ไทย,NN
4,0,เข้า,AV
...,...,...,...
246474,5619,จำเป็น,VV
246475,5619,ก็,CC
246476,5619,พร้อม,VV
246477,5619,ปฏิบัติ,VV


**Create Test Set**

In [ ]:
listfile = os.listdir('/content/LST20_Corpus/test/')
path_to_text = '/content/LST20_Corpus/test/'

sentence_id = []
words = []
labels = []
sent_num = 0
for file_ in tqdm(listfile):
  #print(file_)
  try:
    f = open(path_to_text+file_)
    for line in f:
      if len(line)==1 or line.startswith('-DOCSTART') or line[0]=="\n": #ถ้าเจอบรรทัดว่าง
        sentence_id.append('')
        words.append('')
        labels.append('')
        sent_num += 1
      else:
        splits = line.split('\t')
        sentence_id.append(sent_num)
        words.append(splits[0])
        labels.append(splits[1])
    f.close()
  except:
    #print("Directory")
    pass


test_ = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels})

  0%|          | 0/967 [00:00<?, ?it/s]

In [ ]:
test_ = test_.drop(test_[test_.sentence_id == ''].index)

In [ ]:
test_

,sentence_id,words,labels
0,0,ศาล,NN
1,0,ออก,VV
2,0,หมาย,NN
3,0,จับ,VV
4,0,',PU
...,...,...,...
212522,5249,"""",PU
212523,5249,_,PU
212524,5249,เลขาธิการ,NN
212525,5249,สภาทนายความ,NN


# Create Model and train

** Create Model **

In [ ]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 8
model_args.polynomial_decay_schedule_lr_end = 1e-9
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True
model_args.weight_decay = 1e-2
model_args.num_train_epochs = 5
model_args.max_seq_length = 256
model_args.learning_rate = 4e-5
model_args.wandb_project = 'nlp-POS-tag-hackatron'
model_args.best_model_dir = '/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/' # next iter
model_args.use_early_stopping = False
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.save_model_every_epoch =True
#model_args.lazy_loading = True

# previous iter weight
model = NERModel(
    "camembert", "/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWhole/", args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

# Train the model
#model.train_model( train_data = train_ , eval_data = eval_)
#model.train_model(train_data = '/content/drive/MyDrive/POSTAG/testest_for_lst21.txt' )

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


In [ ]:
model = NERModel(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/', args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


# Predict and submit CSV

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_)

In [ ]:
with open('/content/pos_test.txt') as f:
  text = f.read()
text_split = text.split('\n')
word_list = [word if word != '' else '_' for word in text_split[:-2]]
print(len(word_list))
word_overlap = [' '.join(word_list[int(i*12): int(i*12+60)])for i in range(len(word_list)//12)]

predictions, raw_outputs = model.predict(word_overlap)
from collections import Counter
from tqdm.auto import tqdm
ans_counter = [Counter() for i in range(len(word_list))]
for i in tqdm(range(len(word_list)//12)):
  counts = ans_counter[int(i*12): int(i*12+60)]
  data = predictions[i]
  for each_counter, each_data in zip(counts, data):
    each_counter.update({list(each_data.values())[0]:1})
final = [max(counter, key=counter.get)for counter in ans_counter]

103913


  0%|          | 0/8659 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1083 [00:00<?, ?it/s]

  0%|          | 0/8659 [00:00<?, ?it/s]

In [ ]:
submit = pd.read_csv('/content/pos_sample_submission.csv')
submit['Predicted'] = final
submit.to_csv('submission.csv',index = False)

# Export to ONNX

In [ ]:
!pip install numpy protobuf==3.16.0
!pip install onnx
!pip install -q simpletransformers
!pip install -q datasets transformers[sentencepiece] simpletransformers
!pip install onnxruntime

     |████████████████████████████████| 1.0 MB 14.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


     |████████████████████████████████| 12.8 MB 13.4 MB/s 
     |████████████████████████████████| 4.9 MB 15.6 MB/s 


In [ ]:
import torch
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
from simpletransformers.ner.ner_utils import (
    InputExample,
    LazyNERDataset,
    convert_examples_to_features,
    get_examples_from_df,
    load_hf_dataset,
    read_examples_from_file,
    flatten_results,
)
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import numpy as np

In [ ]:
class CustomNER(NERModel):
  def __init__(self,model_type,
        model_name,
        labels=None,
        weight=None,
        args=None,
        use_cuda=True,
        cuda_device=-1,
        onnx_execution_provider=None,
        **kwargs,):
    super().__init__(model_type,model_name,labels,weight,args,use_cuda,cuda_device,onnx_execution_provider,**kwargs)

  def predict(self, to_predict, split_on_space=True):
        """
        Performs predictions on a list of text.
        Args:
            to_predict: A python list of text (str) to be sent to the model for prediction.
            split_on_space: If True, each sequence will be split by spaces for assigning labels.
                            If False, to_predict must be a a list of lists, with the inner list being a
                            list of strings consisting of the split sequences. The outer list is the list of sequences to
                            predict on.
        Returns:
            preds: A Python list of lists with dicts containing each word mapped to its NER tag.
            model_outputs: A Python list of lists with dicts containing each word mapped to its list with raw model output.
        """  # noqa: ignore flake8"
        #print('predict')
        device = self.device
        model = self.model
        args = self.args
        pad_token_label_id = self.pad_token_label_id
        preds = None
        print(to_predict)
        if split_on_space:
            if self.args.model_type == "layoutlm":
                predict_examples = [
                    InputExample(
                        i,
                        sentence.split(),
                        [self.args.labels_list[0] for word in sentence.split()],
                        x0,
                        y0,
                        x1,
                        y1,
                    )
                    for i, (sentence, x0, y0, x1, y1) in enumerate(to_predict)
                ]
                to_predict = [sentence for sentence, *_ in to_predict]
            else:
                predict_examples = [
                    InputExample(
                        i,
                        sentence.split(),
                        [self.args.labels_list[0] for word in sentence.split()],
                    )
                    for i, sentence in enumerate(to_predict)
                ]
        else:
            if self.args.model_type == "layoutlm":
                predict_examples = [
                    InputExample(
                        i,
                        sentence,
                        [self.args.labels_list[0] for word in sentence],
                        x0,
                        y0,
                        x1,
                        y1,
                    )
                    for i, (sentence, x0, y0, x1, y1) in enumerate(to_predict)
                ]
                to_predict = [sentence for sentence, *_ in to_predict]
            else:
                predict_examples = [
                    InputExample(
                        i, sentence, [self.args.labels_list[0] for word in sentence]
                    )
                    for i, sentence in enumerate(to_predict)
                ]

        if self.args.onnx:

            # Encode
            model_inputs = self.tokenizer.batch_encode_plus(
                to_predict,
                return_tensors="pt",
                padding=True,
                truncation=True,
                is_split_into_words=(not split_on_space),
            )

            # Change shape for batching
            encoded_model_inputs = []
            if self.args.model_type in ["bert", "xlnet", "albert", "layoutlm"]:
                for (input_ids, attention_mask, token_type_ids) in tqdm(
                    zip(
                        model_inputs["input_ids"],
                        model_inputs["attention_mask"],
                        model_inputs["token_type_ids"],
                    )
                ):
                    encoded_model_inputs.append(
                        (input_ids, attention_mask, token_type_ids)
                    )
            else:
                for (input_ids, attention_mask) in tqdm(
                    zip(model_inputs["input_ids"], model_inputs["attention_mask"])
                ):
                    encoded_model_inputs.append((input_ids, attention_mask))

            # Setup batches
            eval_sampler = SequentialSampler(encoded_model_inputs)
            eval_dataloader = DataLoader(
                encoded_model_inputs,
                sampler=eval_sampler,
                batch_size=args.eval_batch_size,
            )
            for batch in tqdm(
                eval_dataloader, disable=args.silent, desc="Running Prediction"
            ):
                if self.args.model_type in ["bert", "xlnet", "albert", "layoutlm"]:
                    inputs_onnx = {
                        "input_ids": batch[0].detach().cpu().numpy(),
                        "attention_mask": batch[1].detach().cpu().numpy(),
                        "token_type_ids": batch[2].detach().cpu().numpy(),
                    }
                else:
                    inputs_onnx = {
                        "input_ids": batch[0].detach().cpu().numpy(),
                        "attention_mask": batch[1].detach().cpu().numpy(),
                    }

                # Run the model (None = get all the outputs)
                output = self.model.run(None, inputs_onnx)

                if preds is None:
                    preds = output[0]
                    out_input_ids = inputs_onnx["input_ids"]
                    out_attention_mask = inputs_onnx["attention_mask"]
                else:
                    preds = np.append(preds, output[0], axis=0)
                    out_input_ids = np.append(
                        out_input_ids, inputs_onnx["input_ids"], axis=0
                    )
                    out_attention_mask = np.append(
                        out_attention_mask, inputs_onnx["attention_mask"], axis=0
                    )
            out_label_ids = np.zeros_like(out_input_ids)
            for index in range(len(out_label_ids)):
                out_label_ids[index][0] = -100
                out_label_ids[index][-1] = -100
        else:

            eval_dataset = self.load_and_cache_examples(
                None, to_predict=predict_examples
            )
            eval_sampler = SequentialSampler(eval_dataset)
            eval_dataloader = DataLoader(
                eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size
            )

            self._move_model_to_device()

            eval_loss = 0.0
            nb_eval_steps = 0
            preds = None
            out_label_ids = None
            model.eval()

            if args.n_gpu > 1:
                model = torch.nn.DataParallel(model)

            if self.args.fp16:
                from torch.cuda import amp

            for batch in tqdm(
                eval_dataloader, disable=args.silent, desc="Running Prediction"
            ):
                batch = tuple(t.to(device) for t in batch)

                with torch.no_grad():
                    inputs = self._get_inputs_dict(batch)

                    if self.args.fp16:
                        with amp.autocast():
                            outputs = self._calculate_loss(
                                model,
                                inputs,
                                loss_fct=self.loss_fct,
                                num_labels=self.num_labels,
                                args=self.args,
                            )
                            tmp_eval_loss, logits = outputs[:2]
                    else:
                        outputs = self._calculate_loss(
                            model,
                            inputs,
                            loss_fct=self.loss_fct,
                            num_labels=self.num_labels,
                            args=self.args,
                        )
                        tmp_eval_loss, logits = outputs[:2]

                    if self.args.n_gpu > 1:
                        tmp_eval_loss = tmp_eval_loss.mean()
                    eval_loss += tmp_eval_loss.item()

                nb_eval_steps += 1

                if preds is None:
                    preds = logits.detach().cpu().numpy()
                    out_label_ids = inputs["labels"].detach().cpu().numpy()
                    out_input_ids = inputs["input_ids"].detach().cpu().numpy()
                    out_attention_mask = inputs["attention_mask"].detach().cpu().numpy()
                else:
                    preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                    out_label_ids = np.append(
                        out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0
                    )
                    out_input_ids = np.append(
                        out_input_ids,
                        inputs["input_ids"].detach().cpu().numpy(),
                        axis=0,
                    )
                    out_attention_mask = np.append(
                        out_attention_mask,
                        inputs["attention_mask"].detach().cpu().numpy(),
                        axis=0,
                    )

            eval_loss = eval_loss / nb_eval_steps
        token_logits = preds
        preds = np.argmax(preds, axis=2)

        label_map = {i: label for i, label in enumerate(self.args.labels_list)}

        out_label_list = [[] for _ in range(out_label_ids.shape[0])]
        preds_list = [[] for _ in range(out_label_ids.shape[0])]

        for i in range(out_label_ids.shape[0]):
            for j in range(out_label_ids.shape[1]):
                if out_label_ids[i, j] != pad_token_label_id:
                    out_label_list[i].append(label_map[out_label_ids[i][j]])
                    preds_list[i].append(label_map[preds[i][j]])

        if split_on_space:
            preds = [
                [
                    {word: preds_list[i][j]}
                    for j, word in enumerate(sentence.split()[: len(preds_list[i])])
                ]
                for i, sentence in enumerate(to_predict)
            ]
        else:
            preds = [
                [
                    {word: preds_list[i][j]}
                    for j, word in enumerate(sentence[: len(preds_list[i])])
                ]
                for i, sentence in enumerate(to_predict)
            ]

        word_tokens = []
        for n, sentence in enumerate(to_predict):
            w_log = self._convert_tokens_to_word_logits(
                out_input_ids[n],
                out_label_ids[n],
                out_attention_mask[n],
                token_logits[n],
            )
            word_tokens.append(w_log)

        if split_on_space:
            model_outputs = [
                [
                    {word: word_tokens[i][j]}
                    for j, word in enumerate(sentence.split()[: len(preds_list[i])])
                ]
                for i, sentence in enumerate(to_predict)
            ]
        else:
            model_outputs = [
                [
                    {word: word_tokens[i][j]}
                    for j, word in enumerate(sentence[: len(preds_list[i])])
                ]
                for i, sentence in enumerate(to_predict)
            ]

        return preds, model_outputs

In [ ]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [ ]:
# Configure the model
model_args = NERArgs()
model_args.max_seq_length = 256
model_args.dynamic_quantize = True
model = NERModel(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/', labels=_POS_TAGS,args=model_args, use_cuda=torch.cuda.is_available())

In [ ]:
# Converting a Simple Transformers model to the ONNX format.
model.convert_to_onnx("/content/drive/MyDrive/POSTAG/BERT_model/ONNXmodel")

ONNX opset version set to: 11
Loading pipeline (model: /tmp/tmpg3hwxb5r, tokenizer: PreTrainedTokenizer(name_or_path='/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']}))


Some weights of the model checkpoint at /tmp/tmpg3hwxb5r were not used when initializing CamembertForTokenClassification: ['roberta.encoder.layer.1.attention.output.dense._packed_params.dtype', 'roberta.encoder.layer.9.output.dense.scale', 'roberta.encoder.layer.1.attention.self.key.scale', 'roberta.encoder.layer.0.output.dense._packed_params.dtype', 'roberta.encoder.layer.6.output.dense.zero_point', 'roberta.encoder.layer.2.output.dense._packed_params.dtype', 'roberta.encoder.layer.1.output.dense.zero_point', 'roberta.encoder.layer.8.intermediate.dense._packed_params.dtype', 'roberta.encoder.layer.9.intermediate.dense._packed_params.dtype', 'roberta.encoder.layer.6.attention.self.key._packed_params._packed_params', 'roberta.encoder.layer.6.output.dense.scale', 'roberta.encoder.layer.8.attention.output.dense.scale', 'roberta.encoder.layer.2.attention.self.key.zero_point', 'roberta.encoder.layer.1.attention.output.dense.zero_point', 'roberta.encoder.layer.2.attention.self.value._packed_

Using framework PyTorch: 1.10.0+cu111
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Ensuring inputs are in correct order
token_type_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask']


In [ ]:
# Test Loading a converted ONNX model
model_args = NERArgs()
model_args.max_seq_length = 256
model_args.dynamic_quantize = True
model = NERModel(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/ONNXmodel', labels=_POS_TAGS, args=model_args, use_cuda=torch.cuda.is_available())

OSError: ignored

In [ ]:
model = CustomNER(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/ONNXmodel', labels=_POS_TAGS)

         Please use quantize_static for static quantization, quantize_dynamic for dynamic quantization.


As of onnxruntime 1.4.0, models larger than 2GB will fail to quantize due to protobuf constraint.
This limitation will be removed in the next release of onnxruntime.
Quantized model has been written at /content/drive/MyDrive/POSTAG/BERT_model/ONNXmodel/onnx_model-quantized.onnx: ✔


/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names)))
/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


In [ ]:
# Test inference 
from time import time
from tqdm.notebook import tqdm

with open('/content/pos_test.txt') as f:
  text = f.read()
text_split = text.split('\n')
word_list = [word if word != '' else '_' for word in text_split[:-2]]
print(len(word_list))
word_overlap = [' '.join(word_list[int(i*12): int(i*12+60)])for i in range(len(word_list)//12)]



103913


In [ ]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 8
model_args.polynomial_decay_schedule_lr_end = 1e-9
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True
model_args.weight_decay = 1e-2
model_args.num_train_epochs = 5
model_args.max_seq_length = 256
model_args.learning_rate = 4e-5
model_args.wandb_project = 'nlp-POS-tag-hackatron'
model_args.best_model_dir = '/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/' # next iter
model_args.use_early_stopping = False
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.save_model_every_epoch =True
model_args.max_seq_length = 256
model_args.dynamic_quantize = True
#args=model_args


model = NERModel(
    "camembert",'/content/onnx_outputs' , labels=_POS_TAGS, use_cuda=False, args=model_args )

OSError: ignored

In [ ]:
start = time()
print(model.predict(word_overlap))
end = time()
print(f"ONNX CPU: {end - start}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0it [00:00, ?it/s]

Running Prediction:   0%|          | 0/1083 [00:00<?, ?it/s]

KeyboardInterrupt: ignored